# Fill Form Agents

graph TD
    A[User Agent] -->|fill form| B[Form Agent]
    B -->|result of saving progress| C[Assistant Agent]
    C -->|generate prompt to ask more information,
    or termnate conversation if all infomration is collected| A


Link `autogen` to `site-packages/hdcola`

In [1]:
from typing import Any, Dict, List, Optional
from hdcola.form.form_schema import read_form_schema

def get_form_schema_tool() -> Optional[List[Dict[str, Any]]]:
    """
    Retrieves the form schema from a JSON file.

    Returns:
        str: A list of dictionaries representing the form schema.
    """
    try:
        schema_path = "./form_schema.json"
        schema = read_form_schema(schema_path)
        return schema
    except Exception as e:
        return f"Error: {str(e)}"

get_form_schema_tool()

[{'name': 'username',
  'type': 'string',
  'required': True,
  'description': '用户名，长度需在3-20个字符之间'},
 {'name': 'age',
  'type': 'number',
  'required': False,
  'description': '年龄，请输入正整数'},
 {'name': 'email',
  'type': 'string',
  'required': True,
  'description': '电子邮箱地址，用于接收通知'}]

In [2]:
from hdcola.form.form_data import read_form_data
from typing import Any, Dict

def get_form_data_tool() -> Dict[str, Any]:
    """
    Reads form data from a JSON file and returns it as a dictionary.

    Returns:
        str: The form data.
    """
    try:
        form_data_path = "./form_data.json"
        form_data = read_form_data(form_data_path)
        return form_data
    except Exception as e:
        return f"Error: {str(e)}"
    
get_form_data_tool()

{'name': 'John Doe'}

In [3]:
from hdcola.form.form_data import update_form_field

def update_form_data_tool(name: str, value: Any) -> bool:
    """
    Update a field in the form data, creates the file if it doesn't exist

    Args:
        name: Name of the field to update
        value: New value for the field

    Returns:
        bool: The updated form data if successful, or an error message if it fails
    """
    try:
        form_data_path = "./form_data.json"
        updated_form_data = update_form_field(form_data_path, name, value)
        return updated_form_data
    except Exception as e:
        return f"Error: {str(e)}"
    
update_form_data_tool("name", "John Doe")

True

In [4]:
import os
from dotenv import load_dotenv
from getpass import getpass

# Load environment variables from .env file
load_dotenv()

# Get API base URL and model with default values
OPENAI_MODEL = os.getenv("OPENAI_MODEL", "gpt-4o")

# Set up OpenAI API configuration
# Try to get API key from environment variables
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# If not found, ask user to input it
if not OPENAI_API_KEY:
    OPENAI_API_KEY = getpass("Enter your OpenAI API key:")

from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.models.ollama import OllamaChatCompletionClient
from autogen_core.models import UserMessage

model_client = OpenAIChatCompletionClient(
    model=OPENAI_MODEL,
    api_key=OPENAI_API_KEY,
)

# model_client = OllamaChatCompletionClient(
#     model=OPENAI_MODEL
# )

OPENAI_MODEL

'gpt-4o-mini'

In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.ui import Console

text_mention_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=15)
termination = text_mention_termination | max_messages_termination

planning_agent = AssistantAgent(
    "PlanningAgent",
    description="An agent for planning tasks, this agent should be the first to engage when given a new task.",
    model_client=model_client,
    system_message="""
    You are a planning agent.
    Your job is to break down complex tasks into smaller, manageable subtasks.
    Your team members are:
        FormAgent: Update form data, read form data, read form schema
        AssistantAgent: As an assistant, generate prompts to ask for more information

    You only plan and delegate tasks - you do not execute them yourself.

    When assigning tasks, use this format:
    1. <agent> : <task>

    After all tasks are complete, summarize the findings and end with "TERMINATE".
    """,
)

form_agent = AssistantAgent(
    "FormAgent",
    description="An agent for get and save the form data or get the form schema.",
    tools=[get_form_schema_tool, get_form_data_tool,update_form_data_tool],
    model_client=model_client,
    system_message="""
    You are form agent.
    You have two tools:
        get_form_schema_tool - use it to get the form schema.
        get_form_data_tool - use it to get the results of what the user has filled in.
        update_form_data_tool - use it to update the form data.
    """,
)

form_analyst_agent = AssistantAgent(
    "FormAnalystAgent",
    description="An agent for analyst from data and schema.",
    model_client=model_client,
    tools=[],
    system_message="""
    You are a form analyst.
    Depending on the task assigned to you, you should analyze the data that has been filled in and the structure of the form to generate the next step to ask the user questions to get more data on the form. 
    If the user has already completed the form, prompt the user to check or end the completion.
    """,
)

selector_prompt = """Select an agent to perform task.

{roles}

Current conversation context:
{history}

Read the above conversation, then select an agent from {participants} to perform the next task.
Make sure the planner agent has assigned tasks before other agents start working.
Only select one agent.
"""

team = SelectorGroupChat(
    [planning_agent, form_analyst_agent],
    model_client=model_client,
    termination_condition=termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True,  # Allow an agent to speak multiple turns in a row.
)

task = "Let's start filling out the forms."
await Console(team.run_stream(task=task))

---------- user ----------
Let's start filling out the forms.
---------- PlanningAgent ----------
To begin filling out the forms, let's first clarify what specific form data we need to collect and any particular details or sections that require special attention. 

1. AssistantAgent : Generate prompts to ask the user for details about the forms they need to fill out, including any specific sections or data points required. 

Once we have this information, we can proceed with updating and organizing the form data.
---------- PlanningAgent ----------
1. AssistantAgent : What type of forms are we filling out?
2. AssistantAgent : What specific data do you need to include in the forms?
3. AssistantAgent : Are there any deadlines or timeframes we need to be aware of for filling out these forms?
4. AssistantAgent : Do you have any existing data or documents that need to be referenced while filling out these forms?
5. AssistantAgent : Is there a specific format or template for the forms that w

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content="Let's start filling out the forms.", type='TextMessage'), TextMessage(source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=135, completion_tokens=80), metadata={}, content="To begin filling out the forms, let's first clarify what specific form data we need to collect and any particular details or sections that require special attention. \n\n1. AssistantAgent : Generate prompts to ask the user for details about the forms they need to fill out, including any specific sections or data points required. \n\nOnce we have this information, we can proceed with updating and organizing the form data.", type='TextMessage'), TextMessage(source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=221, completion_tokens=119), metadata={}, content='1. AssistantAgent : What type of forms are we filling out?\n2. AssistantAgent : What specific data do you need to include in the forms?\n3. AssistantAge

In [7]:
task = "User name is Danny and he is 18 years old."
await Console(team.run_stream(task=task))

---------- user ----------
User name is Danny and he is 18 years old.
---------- PlanningAgent ----------
Based on the information provided, we can now break down the task of filling out the forms with the details we have. However, we still need more information to proceed fully. Let's continue with the requirements:

1. FormAgent : Update form data with the user's name (Danny) and age (18 years old).
2. AssistantAgent : What other specific data or sections do you need to include in the forms aside from the user's name and age?
3. AssistantAgent : Are there any deadlines or timeframes we need to be aware of for filling out these forms?
4. AssistantAgent : Do you have any existing data or documents that need to be referenced while filling out these forms?
5. AssistantAgent : Is there a specific format or template for the forms that we should follow? 

Once we gather this additional information, we can proceed with filling out the forms effectively.
---------- FormAnalystAgent ----------

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='User name is Danny and he is 18 years old.', type='TextMessage'), TextMessage(source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=1614, completion_tokens=177), metadata={}, content="Based on the information provided, we can now break down the task of filling out the forms with the details we have. However, we still need more information to proceed fully. Let's continue with the requirements:\n\n1. FormAgent : Update form data with the user's name (Danny) and age (18 years old).\n2. AssistantAgent : What other specific data or sections do you need to include in the forms aside from the user's name and age?\n3. AssistantAgent : Are there any deadlines or timeframes we need to be aware of for filling out these forms?\n4. AssistantAgent : Do you have any existing data or documents that need to be referenced while filling out these forms?\n5. AssistantAgent : Is there a specific format or te

In [8]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.ui import Console

def search_web_tool(query: str) -> str:
    if "2006-2007" in query:
        return """Here are the total points scored by Miami Heat players in the 2006-2007 season:
        Udonis Haslem: 844 points
        Dwayne Wade: 1397 points
        James Posey: 550 points
        ...
        """
    elif "2007-2008" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2007-2008 is 214."
    elif "2008-2009" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2008-2009 is 398."
    return "No data found."


def percentage_change_tool(start: float, end: float) -> float:
    return ((end - start) / start) * 100

planning_agent = AssistantAgent(
    "PlanningAgent",
    description="An agent for planning tasks, this agent should be the first to engage when given a new task.",
    model_client=model_client,
    system_message="""
    You are a planning agent.
    Your job is to break down complex tasks into smaller, manageable subtasks.
    Your team members are:
        WebSearchAgent: Searches for information
        DataAnalystAgent: Performs calculations

    You only plan and delegate tasks - you do not execute them yourself.

    When assigning tasks, use this format:
    1. <agent> : <task>

    After all tasks are complete, summarize the findings and end with "TERMINATE".
    """,
)

web_search_agent = AssistantAgent(
    "WebSearchAgent",
    description="An agent for searching information on the web.",
    tools=[search_web_tool],
    model_client=model_client,
    system_message="""
    You are a web search agent.
    Your only tool is search_tool - use it to find information.
    You make only one search call at a time.
    Once you have the results, you never do calculations based on them.
    """,
)

data_analyst_agent = AssistantAgent(
    "DataAnalystAgent",
    description="An agent for performing calculations.",
    model_client=model_client,
    tools=[percentage_change_tool],
    system_message="""
    You are a data analyst.
    Given the tasks you have been assigned, you should analyze the data and provide results using the tools provided.
    If you have not seen the data, ask for it.
    """,
)

text_mention_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=25)
termination = text_mention_termination | max_messages_termination

selector_prompt = """Select an agent to perform task.

{roles}

Current conversation context:
{history}

Read the above conversation, then select an agent from {participants} to perform the next task.
Make sure the planner agent has assigned tasks before other agents start working.
Only select one agent.
"""

team = SelectorGroupChat(
    [planning_agent, web_search_agent, data_analyst_agent],
    model_client=model_client,
    termination_condition=termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True,  # Allow an agent to speak multiple turns in a row.
)

task = "Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?"
await Console(team.run_stream(task=task))


---------- user ----------
Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?
---------- PlanningAgent ----------
To address your request, I will break it down into the following tasks:

1. WebSearchAgent : Find out which Miami Heat player scored the highest points during the 2006-2007 season.
2. WebSearchAgent : Obtain the total rebounds for the identified player during the 2007-2008 and 2008-2009 seasons.
3. DataAnalystAgent : Calculate the percentage change in total rebounds between the 2007-2008 and 2008-2009 seasons.

Let's proceed with these tasks.
---------- WebSearchAgent ----------
[FunctionCall(id='call_zGaFyaBbFCgeBCqS2xHu546e', arguments='{"query":"Miami Heat player highest points 2006-2007 season"}', name='search_web_tool')]
---------- WebSearchAgent ----------
[FunctionExecutionResult(content='Here are the total points scored by Miami Heat play

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), TextMessage(source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=161, completion_tokens=110), metadata={}, content="To address your request, I will break it down into the following tasks:\n\n1. WebSearchAgent : Find out which Miami Heat player scored the highest points during the 2006-2007 season.\n2. WebSearchAgent : Obtain the total rebounds for the identified player during the 2007-2008 and 2008-2009 seasons.\n3. DataAnalystAgent : Calculate the percentage change in total rebounds between the 2007-2008 and 2008-2009 seasons.\n\nLet's proceed with these tasks.", type='TextMessage'), ToolCallRequestEvent(source='WebSearchAgent', models_usage=RequestUsage(prompt_tokens=258, completion_to